In [1]:
import pandas as pd
import numpy as np
import json 
from tqdm import tqdm

f = open("../data/sample_sim.json") # whatever the name of your sim json file is
sims = json.load(f)

print(sims.keys())
# SimulationResults has the goods in it

def to_df(x, name=None):
    """
    Simple function that converts
    different objects to a pandas
    DataFrame. (Probably bad code)
    """
    try:
        out = pd.DataFrame(x)
    except:
        try:
            out = pd.DataFrame(x, index = [0])
            if name is not None:
                out.columns = [f"{name}_{c}" for c in out.columns]
        except:
            out = pd.DataFrame({name:x},index=[0])
    return out

def parse_sim_results(sims, SimIndex):
    """
    Function to process a single simulation
    """
    # pull out simulation result
    result = sims["SimulationResults"][SimIndex]
    
    result_dict = {"SimResults": pd.DataFrame({"SimIndex":SimIndex},index=[0])}
    for key in result.keys():
        element_df = to_df(result[key], key)
        if element_df.shape[0] == 1:
            result_dict["SimResults"] = pd.concat([result_dict["SimResults"], element_df], axis = 1)
        else:
            if "SimIndex" not in element_df.columns:
                element_df["SimIndex"] = SimIndex
            result_dict[key] = element_df

    return result_dict

def process_simulation_results(sims):
    """
    Function to process all of the sims
    """
    n_sims = sims["NumberOfSimulations"]
    print(f"Parsing results for {n_sims} of simulations")

    sim_results = {}
    for i in tqdm(range(n_sims)):
        sim_i_result = parse_sim_results(sims, i)
        for key in sim_i_result.keys():
            if key in sim_results.keys():
                sim_results[key] = pd.concat([sim_results[key], sim_i_result[key]], axis=0)
            else:
                sim_results[key] = sim_i_result[key]

    return sim_results


# Process the Simulation Results
SimulationResults = process_simulation_results(sims)


dict_keys(['GameId', 'Solution', 'MarketIdentifiers', 'BatchingEnabled', 'HomePlayers', 'AwayPlayers', 'SimulationResults', 'NextPlayProbabilities', 'EventId', 'NumberOfSimulations', 'HomeTeamStats', 'AwayTeamStats', 'ModelInputs'])
Parsing results for 10000 of simulations


100%|██████████| 10000/10000 [01:29<00:00, 111.31it/s]


In [6]:
sims["EventId"]

10197739

In [2]:
SimulationResults.keys()

dict_keys(['SimResults', 'ScoringPlays', 'PlayerYards'])

In [26]:
# pull out id tables
AwayPlayers = pd.DataFrame(sims["AwayPlayers"])
HomePlayers = pd.DataFrame(sims["HomePlayers"])

In [4]:

name = 'Brock Purdy'
pd.DataFrame(pd.concat([HomePlayers, AwayPlayers],axis=0).query(f"PlayerName=='{name}'")["Propositions"].values[0]).sort_values("Line",ascending=False)

,PropositionIndex,PropositionType,Line,OverProbability
7,0,12,245.5,0.500000
0,1,1,236.5,0.500000
4,0,4,37.5,0.500000
3,0,24,30.5,0.500000
10,3,17,25.0,0.285714
6,0,2,20.5,0.465116
8,2,17,20.0,0.454545
9,1,19,15.0,0.333333
5,5,5,6.5,0.526316
1,0,3,1.5,0.454545


Proposition Types:
- 1 = passing yards
- 5 = rushing yards
- 6 = rushing att -- > not certain
- 8 = recieving yards
- 9 = Receptions
- 13 = Total Yards

## Any Player to Rush for 100 Yards

In [7]:
def any_player_to_rush_for_x_yards(SimulationResults, yards):

    plr_yards = SimulationResults["PlayerYards"]\
        .query("PropositionType==5").copy()
        
    plr_yards["over_x"] = [1 if x > yards else 0 for x in plr_yards["YardsGained"]]

    yards_sim = plr_yards.groupby("SimIndex").agg({"over_x":"sum"}).reset_index()
    yards_sim["over_x"] = [1 if x > 0 else 0 for x in yards_sim["over_x"]]

    return np.mean(yards_sim["over_x"])

any_player_to_rush_for_x_yards(SimulationResults, 100)

0.2874

## Will Team X Rush for  TD?

In [8]:
def teams_to_rush_for_td(SimulationResults):

    rush_tds = SimulationResults["SimResults"]\
        .assign(
            HomeRushingTouchdowns = lambda x: (
                x.HomeTouchdowns_FirstQuarter + x.HomeTouchdowns_SecondQuarter +
                x.HomeTouchdowns_ThirdQuarter + x.HomeTouchdowns_FourthQuarter + 
                x.HomeTouchdowns_Overtime - x.HomePassingTouchdowns
            ),
            AwayRushingTouchdowns = lambda x: (
                x.AwayTouchdowns_FirstQuarter + x.AwayTouchdowns_SecondQuarter +
                x.AwayTouchdowns_ThirdQuarter + x.AwayTouchdowns_FourthQuarter + 
                x.AwayTouchdowns_Overtime - x.AwayPassingTouchdowns
            )
        ).copy()
    rush_tds["HomeRushingTouchdown"] = [1 if x > 0 else 0 for x in rush_tds["HomeRushingTouchdowns"]]
    rush_tds["AwayRushingTouchdown"] = [1 if x > 0 else 0 for x in rush_tds["AwayRushingTouchdowns"]]

    return rush_tds.agg({"HomeRushingTouchdown":"mean","AwayRushingTouchdown":"mean"})

teams_to_rush_for_td(SimulationResults)

HomeRushingTouchdown    0.7328
AwayRushingTouchdown    0.6034
dtype: float64

## Yards Head to Head

In [9]:
plr1 = "Ezekiel Elliott"
plr2 = "Tony Pollard"

def head_to_head_yards(sims, SimulationResults, plr1, plr2, prop):

    prop_dict = {
        "receiving": 8,
        "rushing": 5,
        "passing": 1
    }


    assert prop in prop_dict.keys(), "prop must be in ['receiving', 'rushing', 'passing']"

    # pull out id tables
    AwayPlayers = pd.DataFrame(sims["AwayPlayers"])
    HomePlayers = pd.DataFrame(sims["HomePlayers"])

    yards_df = pd.concat([
        HomePlayers[["PlayerId", "PlayerName", "Position"]]\
            .merge(
                SimulationResults["PlayerYards"],
                on = "PlayerId"
            ),
        AwayPlayers[["PlayerId", "PlayerName", "Position"]]\
            .merge(
                SimulationResults["PlayerYards"],
                on = "PlayerId"
            )
    ],axis=0)\
        .query(f"PropositionType == {prop_dict[prop]}")

    yards_df = pd.merge(
        yards_df.query(f"PlayerName=='{plr1}'")[["SimIndex", "YardsGained"]],
        yards_df.query(f"PlayerName=='{plr2}'")[["SimIndex", "YardsGained"]],
        on = "SimIndex", suffixes=["_plr1", "_plr2"]
    )

    yards_df["total"] = yards_df["YardsGained_plr1"] + yards_df["YardsGained_plr2"]
    yards_df["plr1_diff"] = yards_df["YardsGained_plr1"] - yards_df["YardsGained_plr2"]
    yards_df["plr1_more"] = yards_df.apply(lambda x: 1 if x.YardsGained_plr1 > x.YardsGained_plr2 else 0, axis = 1)

    total = np.round(np.median(yards_df["total"]) * 2)/2
    plr_1_spread = np.round( -1*np.median(yards_df["plr1_diff"]) * 2)/2
    p_plr1_more = np.mean(yards_df["plr1_more"])

    ## add on juice


    return {
        "Player1": plr1,
        "Player2": plr2,
        "Player1 Prob": p_plr1_more,
        "Spread": plr_1_spread,
        "Total": total,
    }


plr1 = "Christian McCaffrey"
plr2 = "Tony Pollard"
head_to_head_yards(sims, SimulationResults, plr1, plr2, "rushing")


{'Player1': 'Christian McCaffrey',
 'Player2': 'Tony Pollard',
 'Player1 Prob': 0.6743,
 'Spread': -20.5,
 'Total': 118.0}

In [74]:
SimulationResults["PlayerYards"].PropositionType.unique()

array([1, 5, 8])

In [76]:
SimulationResults["ScoringPlays"]

,ScoreType,ScoringSide,Period,SecondsElapsedInQuarter,ScoringPlayerIndex,Method,ScoredByPassingPlayerIndex,ScoredByPassingPlayerId,SimIndex
0,1,1,1,283,NaN,0,NaN,NaN,0
1,0,2,1,559,7.0,1,0.0,1519.0,0
2,2,2,1,564,NaN,0,NaN,NaN,0
3,1,2,1,759,NaN,0,NaN,NaN,0
4,1,1,2,52,NaN,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
7,0,2,4,179,14.0,2,NaN,NaN,9999
8,2,2,4,184,NaN,0,NaN,NaN,9999
9,1,1,4,438,NaN,0,NaN,NaN,9999
10,0,2,4,745,0.0,1,NaN,NaN,9999
